In [105]:
!pip install -q transformers
!pip install -U tensorflow
import pandas as pd
import json
import numpy as np
from google.colab import drive


import numpy as np
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds


import sklearn as sk
import os
import nltk
from nltk.data import find

import matplotlib.pyplot as plt

import re
from sklearn.model_selection import train_test_split
import string
import ast
from keras import backend as K


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [106]:
from transformers import RobertaTokenizerFast, TFRobertaModel, AutoModelForSequenceClassification, AutoTokenizer
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')
model = TFRobertaModel.from_pretrained('roberta-base',output_hidden_states=True, output_attentions=True)



Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [107]:
tokenizer2 = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model2 = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [108]:

drive.mount('/content/gdrive')
sentfin = pd.read_csv('/content/gdrive/MyDrive/Raw Data/SEntFiN-v1.1.csv')
from keras.layers import Input, Flatten
from sklearn.model_selection import train_test_split

X = np.arange(0,10753)
y = np.arange(0,10753)
x_train, x_test, y_train, y_test = train_test_split(sentfin['Title'], sentfin['Decisions'], test_size=0.3,  random_state=8)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.666000, random_state=8)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [109]:
### Function Helps catch all special cases in the tokenization function

def return_helper(ex_title, ex):
    try: 
      return_index = ex_title.index(ex+ ":") 
    except: 
      try: 
        return_index = ex_title.index(ex+ ",")
      except:
        try: 
          return_index = ex_title.index(ex+ "'s")
        except:
          try: 
            return_index = ex_title.index(ex+ ";")
          except: 
            try: 
              return_index = ex_title.index(ex+ "?")
            except:
              try:
                return_index = ex_title.index(ex+ "'")
              except:
                try:
                  return_index = ex_title.index(ex+ "s")
                except:
                  try:
                    return_index = ex_title.index(ex + "s'")
                    
                  except:
                    try:
                      return_index = ex_title.index(ex + 's:')
                      
                    except:
                      return None
    return return_index



In [110]:
def clean_data2(x_data,y_data, tokenizer= tokenizer):
  '''input: news headline data set with sentiments
  output: roberta tokenized inputs, and corresponding label
  '''

  title =[str(i) for i in x_data.values]
  headlines = []
  delete = []

  for row in title:
    headline = " ".join("".join([" " if ch in string.punctuation else ch for ch in row]).split())
    headlines.append(headline)
  try:
    decisions = ([json.loads(i) for i in y_data])
  except:
    decisions = (([ast.literal_eval(i) for i in y_data]))
  entities = []
  sents = []
  for ex in decisions:
    r_entities = []
    r_sents = []
    for key in ex.keys():
      r_entities.append(" ".join("".join([" " if ch in string.punctuation else ch for ch in key]).split()))
    for value in ex.values():
      r_sents.append(value)
    entities.append(r_entities)
    sents.append(r_sents)

  title_tokenizer = tokenizer(headlines, padding='max_length', max_length=30, truncation=True, return_tensors="tf")
  text_tokenized = tokenizer.batch_encode_plus(headlines, padding='max_length', max_length=30, truncation=True, return_tensors="tf")

  problems = 0 
  i = 0
  fin_labels = []
  while i < len(y_data):
    

    ex_title = headlines[i].split()
    
    
    indexes = []

    input = np.array(text_tokenized.word_ids(i))

    for ex_entity in entities[i]:
      word_ent = []
      ex = ex_entity.split()[0]
      try:
        return_index = ex_title.index(ex)
      
        matches = [j for j,val in enumerate(input) if val==return_index]
        
        for matching in matches:
          word_ent.append(matching) 
    
      except:
        helper = return_helper(ex_title, ex)
        if helper is None:
          problems = problems + 1
          
          delete.append(i)
          continue
        return_index = helper
        matches = [j for j,val in enumerate(input) if val==return_index]
        for matching in matches:
          word_ent.append(matching)
      if len(ex_entity.split()) > 1:
        for ex in ex_entity.split()[1:]:
          
          return_index = return_index + 1
          matches = [j for j,val in enumerate(input) if val==return_index]
          for matching in matches:
              word_ent.append(matching)


 
      indexes.append(word_ent)
 
    return_label = np.zeros(30)
    z = 0
    while z < len(indexes):
      x = 0 
      while x < len(indexes[z]):
        if x == 0:
          
          if sents[i][z] == 'positive':
            return_label[indexes[z][x]] = 4
          elif sents[i][z] == 'neutral':
            return_label[indexes[z][x]] = 3
          else: 
            return_label[indexes[z][x]] = 2
        else: 
          return_label[indexes[z][x]] = 1
        x = x + 1
      z = z + 1

    
    
    fin_labels.append(return_label)
    i = i+1

  
  labels = tf.convert_to_tensor(fin_labels, dtype=tf.float32)
  

  dataset = tf.data.Dataset.from_tensor_slices((
      dict(title_tokenizer),
      labels
  ))

  return dataset, dict(title_tokenizer), labels, delete,  title , headlines, decisions

In [111]:
train_dataset, train_tokens, trained_labels,train_delete, train_sentence, train_headline, train_decision = clean_data2(x_train,y_train)


test_dataset, test_tokens, test_labels, test_delete, test_sentence, test_headline, test_decision= clean_data2(x_test,y_test)
val_dataset, val_tokens, valid_labels,val_delete, val_sentence, val_headline, val_decision  = clean_data2(x_val, y_val)

from keras.utils.np_utils import to_categorical   

train_labels = to_categorical(trained_labels, num_classes=5)
train_labels = tf.convert_to_tensor(train_labels )

val_labels = to_categorical(valid_labels, num_classes=5)
val_labels = tf.convert_to_tensor(val_labels )

test_labels = to_categorical(test_labels, num_classes=5)
test_labels = tf.convert_to_tensor(test_labels )

In [112]:
roberta_val = pd.read_csv('/content/gdrive/MyDrive/Raw Data/predictions/base_valF.csv')
roberta_test = pd.read_csv('/content/gdrive/MyDrive/Raw Data/predictions/base_testF.csv')


pt_val = pd.read_csv('/content/gdrive/MyDrive/Raw Data/predictions/base_pt_valF.csv')
pt_test = pd.read_csv('/content/gdrive/MyDrive/Raw Data/predictions/base_pt_testF.csv')

crf_val = pd.read_csv('/content/gdrive/MyDrive/Raw Data/predictions/crf_valF.csv')
crf_test = pd.read_csv('/content/gdrive/MyDrive/Raw Data/predictions/crf_testF.csv')

In [113]:
val_labels_cleaned = []
for example in val_labels.numpy():
  trial = []
  for row in example:
    trial.append(tf.argmax(row, 0, name=None).numpy())
  val_labels_cleaned.append(trial)


test_labels_cleaned = []
for example in test_labels.numpy():
  trial = []
  for row in example:
    trial.append(tf.argmax(row, 0, name=None).numpy())
  test_labels_cleaned.append(trial)


In [114]:
test_labels_cleaned_2 = np.array(np.array(test_labels_cleaned).flatten())


In [115]:
from sklearn.metrics import classification_report
def clean_predictions(predictions, labels, name = '',label_type = '' ):
  print('Results for ' + str(name) + ' on ' + str(label_type))
  y_pred = []
  for prediction in predictions:
    cleaned = prediction.strip('[').strip(']').split()
    cleaned = [int(x) for x in cleaned]
    y_pred.append(cleaned)



  count = 0
  incorrect = 0 
  correct = 0
  correct_pred = []
  

  while count < len(labels):

    if labels[count] != y_pred[count]:
      incorrect = incorrect + 1
      correct_pred.append(0)
      
    if labels[count] == y_pred[count]:
      correct = correct +1

      correct_pred.append(1)
    count = count + 1
  print('Correct Examples: ' + str(correct))


  flatten_pred = np.array(y_pred).flatten()


  flatten_labels = np.array(labels).flatten()
  

  target_names = ['class 0', 'words', 'negative','nuetral','positive']

  print("Classification Report: Example for " + str(name) + ' on ' + str(label_type) +' set')
  #print(classification_report(flatten_labels, flatten_pred, target_names=target_names))
  class_report = classification_report(flatten_labels, flatten_pred, target_names=target_names,digits = 4, output_dict = True)
  print()
  print()
  return correct, class_report, y_pred, correct_pred



In [116]:
#val_review = clean_predictions(roberta_val['predictions'], val_labels_cleaned, 'Base', 'Validation')
#pt_val_review = clean_predictions(pt_val['predictions'], val_labels_cleaned, 'PreTrain', 'Validation')
#crf_val_review = clean_predictions(crf_val['predictions'], val_labels_cleaned, 'CRF', 'Validation')


In [117]:
test_review = clean_predictions(roberta_test['predictions'], test_labels_cleaned, 'OLD MODEL: Disregard', 'Test')
pt_test_review = clean_predictions(pt_test['predictions'], test_labels_cleaned, 'OLD MODEL', 'Test')
crf_test_review = clean_predictions(crf_test['predictions'], test_labels_cleaned, 'OLD MODEL', 'Test')


Results for OLD MODEL: Disregard on Test
Correct Examples: 728
Classification Report: Example for OLD MODEL: Disregard on Test set


Results for OLD MODEL on Test
Correct Examples: 739
Classification Report: Example for OLD MODEL on Test set


Results for OLD MODEL on Test
Correct Examples: 763
Classification Report: Example for OLD MODEL on Test set




In [118]:
test_labels_cleaned_2 = np.array(np.array(test_labels_cleaned).flatten())
test_to_drop = np.array(test_review[2]).flatten()
pt_to_drop = np.array(pt_test_review[2]).flatten()
crf_to_drop = np.array(crf_test_review[2]).flatten()

i = 0
to_delete = [] 
while i < len(test_labels_cleaned_2):
  if test_labels_cleaned_2[i] < 2:
    to_delete.append(i)
  i= i +1
to_delete = np.array(to_delete).flatten()
len(to_delete)
to_delete

test_labels_cleaned_3 = np.delete(test_labels_cleaned_2, to_delete)
crf_to_drop_2 = np.delete(crf_to_drop, to_delete)
pt_to_drop_2 = np.delete(pt_to_drop, to_delete)
test_to_drop_2 = np.delete(test_to_drop, to_delete)
len(test_labels_cleaned_3), len(test_to_drop_2)



(1456, 1456)

In [119]:
def lite_classification(predictions, labels, name = '',label_type = '' ):
  target_names = ['class 0', 'words',  'negative','nuetral','positive']


  print("Classification Report: Example for " + str(name) + ' on ' + str(label_type) +' set')
  print(classification_report(labels,predictions, target_names=target_names))
  class_report = classification_report(labels,predictions,  target_names=target_names,digits = 4, output_dict = True,zero_division = "warn")
  print()
  print()
  return class_report

In [120]:
test_review2 = lite_classification(test_to_drop_2, test_labels_cleaned_3, 'Base', 'Test')
pt_test_review2 = lite_classification(pt_to_drop_2, test_labels_cleaned_3, 'PreTrain', 'Test')
crf_test_review2 = lite_classification(crf_to_drop_2, test_labels_cleaned_3, 'CRF', 'Test')

Classification Report: Example for Base on Test set
              precision    recall  f1-score   support

     class 0       0.00      0.00      0.00         0
       words       0.00      0.00      0.00         0
    negative       0.86      0.85      0.85       402
     nuetral       0.87      0.74      0.80       580
    positive       0.89      0.84      0.86       474

    accuracy                           0.80      1456
   macro avg       0.52      0.49      0.50      1456
weighted avg       0.87      0.80      0.84      1456



Classification Report: Example for PreTrain on Test set
              precision    recall  f1-score   support

     class 0       0.00      0.00      0.00         0
       words       0.00      0.00      0.00         0
    negative       0.86      0.85      0.86       402
     nuetral       0.90      0.74      0.81       580
    positive       0.87      0.88      0.87       474

    accuracy                           0.82      1456
   macro avg       0.

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMe

In [121]:
def sudo_predictions(y_pred, labels):

  count = 0
  incorrect = 0 
  correct = 0
  correct_pred = []

  while count < len(labels):

    
    inc = 0
    token_correct = True
    while inc < 30:

      if labels[count][inc] == 0:
        if y_pred[count][inc] > 1:
          token_correct = False
          
    
      if labels[count][inc] == 1:
          if y_pred[count][inc] > 1:
            token_correct = False 
            

      if (labels[count][inc] > 1):
        if labels[count][inc] != y_pred[count][inc]:
          #print(str(labels[count][inc] )+ str(y_pred[count][inc] ))
          token_correct = False
      inc = inc +1
    if token_correct:
      correct_pred.append(1)
    else:
      correct_pred.append(0)
      



      
    count = count + 1
  print(sum(correct_pred))
  return correct_pred

In [122]:
test_adj = sudo_predictions(test_review[2],test_labels_cleaned)
pt_adj = sudo_predictions(pt_test_review[2],test_labels_cleaned)
crf_adj = sudo_predictions(crf_test_review[2],test_labels_cleaned)

780
782
804


In [124]:
#print(sudo_predictions(val_review[2],val_labels_cleaned), sudo_predictions(val_review[2],val_labels_cleaned)/2149)
#print(sudo_predictions(pt_val_review[2],val_labels_cleaned),sudo_predictions(pt_val_review[2],val_labels_cleaned)/2149) 
#print(sudo_predictions(crf_val_review[2],val_labels_cleaned), sudo_predictions(crf_val_review[2],val_labels_cleaned)/2149) 

In [125]:
def weighted_f1_score(score):
  neg = score['negative']['f1-score']*score['negative']['support']
  pos = score['positive']['f1-score']*score['positive']['support']
  neu = score['nuetral']['f1-score']*score['nuetral']['support']
  support = score['nuetral']['support']+ score['positive']['support'] + score['negative']['support']
  return (neg + pos + neu)/support

In [126]:
print(weighted_f1_score(crf_test_review[1]))
print(weighted_f1_score(pt_test_review[1]))
print(weighted_f1_score(test_review[1]))

0.8166801613726925
0.8140735315472858
0.8103816987513027


In [127]:
crf_test_review[1]

{'class 0': {'precision': 0.9924399592109427,
  'recall': 0.9929287598944591,
  'f1-score': 0.9926842993809791,
  'support': 28425},
 'words': {'precision': 0.9445593711212247,
  'recall': 0.9398929600658708,
  'f1-score': 0.9422203879488237,
  'support': 2429},
 'negative': {'precision': 0.8413098236775819,
  'recall': 0.8308457711442786,
  'f1-score': 0.8360450563204005,
  'support': 402},
 'nuetral': {'precision': 0.7775891341256367,
  'recall': 0.7896551724137931,
  'f1-score': 0.7835757057313942,
  'support': 580},
 'positive': {'precision': 0.8461538461538461,
  'recall': 0.8354430379746836,
  'f1-score': 0.840764331210191,
  'support': 474},
 'accuracy': 0.9809656453110492,
 'macro avg': {'precision': 0.8804104268578465,
  'recall': 0.8777531402986171,
  'f1-score': 0.8790579561183577,
  'support': 32310},
 'weighted avg': {'precision': 0.9809571563916026,
  'recall': 0.9809656453110492,
  'f1-score': 0.9809591719959972,
  'support': 32310}}

In [128]:
data = x_test.reset_index()

In [129]:
data['PT'] = pt_test_review[3]
data['CRF'] = crf_test_review[3]
data['BASE'] = test_review[3]
data['CRF_adj'] = crf_adj

In [130]:
data['SUM'] = data['BASE']+data['CRF']+data['PT']

In [131]:
to_merge = y_test.reset_index()
data = data.merge(to_merge, on = 'index')

In [132]:
data.to_csv('/content/gdrive/MyDrive/Raw Data/predictions/analysis2.csv')

In [133]:
train_dataset, train_tokens, trained_labels,train_delete, train_sentence, train_headline, train_decision = clean_data2(x_train,y_train,tokenizer2)


test_dataset, test_tokens, test_labels, test_delete, test_sentence, test_headline, test_decision= clean_data2(x_test,y_test,tokenizer2)
val_dataset, val_tokens, valid_labels,val_delete, val_sentence, val_headline, val_decision  = clean_data2(x_val, y_val,tokenizer2)
 

train_labels = to_categorical(trained_labels, num_classes=5)
train_labels = tf.convert_to_tensor(train_labels )

val_labels = to_categorical(valid_labels, num_classes=5)
val_labels = tf.convert_to_tensor(val_labels )

test_labels = to_categorical(test_labels, num_classes=5)
test_labels = tf.convert_to_tensor(test_labels )


In [134]:
finbert_v = pd.read_csv('/content/gdrive/MyDrive/Raw Data/predictions/finbert_val.csv')
finbert_t = pd.read_csv('/content/gdrive/MyDrive/Raw Data/predictions/finbert_test.csv')

In [135]:
val_labels_cleaned = []
for example in val_labels.numpy():
  trial = []
  for row in example:
    trial.append(tf.argmax(row, 0, name=None).numpy())
  val_labels_cleaned.append(trial)


test_labels_cleaned = []
for example in test_labels.numpy():
  trial = []
  for row in example:
    trial.append(tf.argmax(row, 0, name=None).numpy())
  test_labels_cleaned.append(trial)

In [136]:
finbert_val_review = clean_predictions(finbert_v['predictions'], val_labels_cleaned, 'Finbert', 'Validation')
finbert_test_review = clean_predictions(finbert_t['predictions'], test_labels_cleaned, 'Fibert', 'Test')


Results for Finbert on Validation
Correct Examples: 1421
Classification Report: Example for Finbert on Validation set


Results for Fibert on Test
Correct Examples: 702
Classification Report: Example for Fibert on Test set




In [137]:
test_labels_cleaned_2 = np.array(np.array(test_labels_cleaned).flatten())
fintest_to_drop = np.array(finbert_test_review[2]).flatten()

i = 0
to_delete = [] 
while i < len(test_labels_cleaned_2):
  if test_labels_cleaned_2[i] < 2:
    to_delete.append(i)
  i= i +1
to_delete = np.array(to_delete).flatten()
len(to_delete)
to_delete

test_labels_cleaned_3 = np.delete(test_labels_cleaned_2, to_delete)
fintest_to_drop_2 = np.delete(fintest_to_drop, to_delete)
lite_classification(fintest_to_drop_2, test_labels_cleaned_3, 'Base', 'Test')


Classification Report: Example for Base on Test set
              precision    recall  f1-score   support

     class 0       0.00      0.00      0.00         0
       words       0.00      0.00      0.00         0
    negative       0.90      0.79      0.84       402
     nuetral       0.83      0.75      0.79       580
    positive       0.83      0.81      0.82       474

    accuracy                           0.78      1456
   macro avg       0.51      0.47      0.49      1456
weighted avg       0.85      0.78      0.81      1456





/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMe

{'class 0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 0},
 'words': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 0},
 'negative': {'precision': 0.8980169971671388,
  'recall': 0.7885572139303483,
  'f1-score': 0.8397350993377484,
  'support': 402},
 'nuetral': {'precision': 0.8336520076481836,
  'recall': 0.7517241379310344,
  'f1-score': 0.7905711695376246,
  'support': 580},
 'positive': {'precision': 0.8311688311688312,
  'recall': 0.810126582278481,
  'f1-score': 0.8205128205128205,
  'support': 474},
 'accuracy': 0.7809065934065934,
 'macro avg': {'precision': 0.5125675671968308,
  'recall': 0.47008158682797274,
  'f1-score': 0.49016381787763874,
  'support': 1456},
 'weighted avg': {'precision': 0.850614713785139,
  'recall': 0.7809065934065934,
  'f1-score': 0.8138927645526607,
  'support': 1456}}